<a href="https://colab.research.google.com/github/shahnawazsyed/MAT422/blob/main/HW14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.4 Principal Component Analysis**
*   Singular value decomposition
*   Low-rank matrix approximations
*   Principal component analysis



In [1]:
import numpy as np

# **1.4.1 Singular Value Decomposition**

In [59]:
A = np.random.randint(10, size=(3,3))
print(A)

[[8 5 6]
 [3 3 9]
 [1 7 2]]


The singular values (σ<sub>1</sub>...σ<sub>n</sub>) of *A* are the square roots of the eigenvalues of *A*<sup>T</sup>*A*:

In [3]:
eigenvalues = np.linalg.eig(np.transpose(A)@A)
eigenvectors = eigenvalues[1]
eigenvalues = eigenvalues[0]
print("Eigenvalues: ", (eigenvalues))

Eigenvalues:  [243.93792934  20.05584078   4.00622987]


In [81]:
SV = np.sqrt(eigenvalues)
SV = np.sort(SV)[::-1]
print("Singular values: ", SV)

Singular values:  [15.61851239  4.47837479  2.00155686]


Let *A* be an *m* x *n* matrix with the dimension of Col(*A*) = *r*. There then exists an *m* x *n* matrix Σ, where the diagonal entries are the first *r* singular values of *A*, arranged in decreasing order. Additionally, there exists an *m* x *m* orthogonal matrix *U* and an *n* x *n* orthogonal matrix V where:

A = UΣV<sup>T</sup>

This is the Singular Value Decomposition (SVD) of *A*. Using the singular values obtained before, we can construct the matrix Σ:

In [82]:
sigma = np.diag(SV, 0)
print(sigma)

[[15.61851239  0.          0.        ]
 [ 0.          4.47837479  0.        ]
 [ 0.          0.          2.00155686]]


The matrix V has columns corresponding to the normalized eigenvectors of *A*<sup>T</sup>*A*:

In [6]:
V = (eigenvectors[:, np.argsort(eigenvalues)[::-1]])
print(V)

[[ 0.46873299  0.87465264 -0.12358052]
 [ 0.35021045 -0.31244377 -0.88302408]
 [ 0.81095131 -0.37062333  0.45276519]]


Finally, we form the matrix U, by re-arranging the SVD equation and evaluating AV/Σ

In [7]:
U = (A @ V) @ np.linalg.inv(sigma)
print(U)

[[ 0.36396881 -0.37101207  0.85432825]
 [ 0.72273885  0.69107731 -0.00779136]
 [ 0.58751618 -0.62029203 -0.51967542]]


We verify the condition of SVD: A = UΣV<sup>T</sup>

In [8]:
print(np.allclose(A, U@sigma@V.transpose(), atol=1e-8))

True


# **1.4.2 Low-Rank Matrix Adaptation**

Suppose we wish to approximate a matrix *A* with a rank-*k* matrix $\hat{A}$, for a target rank *k*.

For computing applications, this is desirable as a compressed version of *A*. For example, it can greatly reduce the amount of parameters, and thus improve performance, in a machine learning model.

We can use the SVD to compute our approximation. The SVD expresses *A* as a sum of rank-1 matricies.

If we want a rank-*k* matrix, we need:

U<sub>k</sub>, the first *k* columns of U.

V<sub>k</sub><sup>T</sup>, the first *k* rows of V<sup>T</sup>.

Σ<sub>k</sub>, the first *k* rows and columns of Σ.

This forms a rank-*k* approximation of $\hat{A}$ = U<sub>k</sub>Σ<sub>k</sub>V<sub>k</sub><sup>T</sup>.

In [86]:
#For simplicity, a helper method for calculting SVD is defined:
def SVD(A):
  eigenvalues = np.linalg.eig(np.transpose(A)@A)
  eigenvectors = eigenvalues[1]
  eigenvalues = eigenvalues[0]
  SV = np.sqrt(eigenvalues)
  SV = np.sort(SV)[::-1]
  sigma = np.diag(SV, 0)
  V = (eigenvectors[:, np.argsort(eigenvalues)[::-1]])
  U = (A @ V) @ np.linalg.inv(sigma)
  return U, sigma, V #NOTE: V is returned NOT already transposed


In [102]:
A = np.random.randint(50, size=(5,5))
print("A:")
print(A)
k = np.random.randint(1, A.shape[1])
print("Approximate A with a rank-k matrix where k =", k)

A:
[[30 21 16  9 44]
 [22 33 41 45 33]
 [ 2  3  1 19 29]
 [44 11 23 10 12]
 [21 21  7 12 26]]
Approximate A with a rank-k matrix where k = 3


In [109]:
U, Sigma, V = SVD(A)
U_k = U[:, :k]
V_kT = np.transpose(V)[:k, :]
sigma_K = Sigma[:k, :k]
print("U_k:")
print(U_k)
print("V_kT:")
print(V_kT)
print("Sigma_k:")
print(sigma_K)

U_k:
[[ 0.49116124  0.17093299  0.58779767]
 [ 0.65954909 -0.43763865 -0.58638868]
 [ 0.22734137 -0.46387307  0.40679329]
 [ 0.38937291  0.74916838 -0.24544861]
 [ 0.34707387  0.05313042  0.29140419]]
V_kT:
[[ 0.46979002  0.38483134  0.403749    0.40345155  0.55264435]
 [ 0.79835801 -0.08045932  0.05379694 -0.52496011 -0.27869977]
 [ 0.02737153 -0.07475617 -0.56327047 -0.38926062  0.72447558]]
Sigma_k:
[[115.20159422   0.           0.        ]
 [  0.          35.88783527   0.        ]
 [  0.           0.          31.66483879]]


In [110]:
#compute A_hat using formula
A_hat = U_k@sigma_K@V_kT
print(A_hat)

[[31.98883335 19.88976984 12.69128159 12.36287704 43.04468592]
 [22.64799965 31.89166242 40.2911173  46.12742163 32.9157685 ]
 [-0.63416928 10.45526775  2.42313101 14.29156964 28.44543286]
 [42.32502429 15.67992064 23.93489387  7.00865113 11.66580431]
 [20.55865417 14.54367971 11.04841273 11.53861832 28.25015747]]


We can evaluate the validity of our approximation by way of the Frobenius norm. We apply the ℓ<sub>2</sub> norm to ||*A* - $\hat{A}$||:

In [111]:
print("Difference between A and approximation:", np.linalg.norm(A - A_hat, 'fro')) #calculate frobenius norm

Difference between A and approximation: 14.730714646903348


# **1.4.3 Principal Component Analysis**